# Eat Safe, Love

## Notebook Set Up

In [8]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [9]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [10]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [11]:
# review the collections in our database
db.list_collection_names()

['establishments', 'uk_food']

In [12]:
# assign the collection to a variable
establishments = db['establishments']


## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [13]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"There are {count} establishments with a hygiene score of 20.")

# Display the first document in the results using pprint
pprint(establishments.find_one(query))

There are 0 establishments with a hygiene score of 20.
None


In [14]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(establishments.find(query))

# Display the number of rows in the DataFrame
print(f"There are {len(df)} rows in this DataFrame.")

# Display the first 10 rows of the DataFrame
df.head(10)

There are 0 rows in this DataFrame.


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [15]:
establishments.find_one()


{'_id': ObjectId('645cb87fe850134df8f75f0d'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': Decimal128('51.49014200')},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

In [16]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {'LocalAuthorityName' :{'$regex': "London"},
         'RatingValue': {'$gte': 4}
         }
results = list(establishments.find(query))

# Use count_documents to display the number of documents in the result
count2 = establishments.count_documents(query2)
print(f"There are {count2} establishments in London that have a RatingValue greater than or equal to 4.")


# Display the first document in the results using pprint
pprint(establishments.find_one(query2))


There are 0 establishments in London that have a RatingValue greater than or equal to 4.
None


In [17]:
# Convert the result to a Pandas DataFrame

df2 = pd.DataFrame(establishments.find(query2))

# Display the number of rows in the DataFrame

print(f"There are {len(df2)} rows in this DataFrame.")

# Display the first 10 rows of the DataFrame
df.head(10)

There are 0 rows in this DataFrame.


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [18]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query3 = {'RatingValue': {'$eq': 5},
         'geocode.latitude':{'$gte': (latitude - degree_search), '$lte': (latitude + degree_search)},
         'geocode.longitude':{'$gte': (longitude - degree_search), '$lte': (longitude + degree_search)}
         }
sort =  [('scores.Hygiene', 1)] # for sorting, 1 is ascending, -1 is descending

# Print the results
for result in establishments.find(query3).sort(sort).limit(5):
    pprint(result)


In [19]:
# Convert result to Pandas DataFrame
pd.DataFrame(establishments.find(query3).sort(sort).limit(5))


""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [20]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

match_query = {'$match': {'scores.Hygiene': 0}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 
                          'count': {'$sum': 1}
                         }
              }
sort_query = {'$sort': {'count': -1}}

pipeline = [match_query, group_query, sort_query]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f"There are {len(results)} documents in the result.")

# Print the first 10 results
pprint(results[0:10])

There are 0 documents in the result.
[]


In [21]:
# Convert the result to a Pandas DataFrame
agg_df = pd.json_normalize(results)

# Display the number of rows in the DataFrame
print(f"There are {len(agg_df)} rows in the DataFrame.")
# Display the first 10 rows of the DataFrame
agg_df.head(10)

There are 0 rows in the DataFrame.


""
